In [5]:
!pip install lmfit

In [6]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import optuna
import wandb
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.ndimage.measurements import center_of_mass
from torch.optim import SGD
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from IPython.display import clear_output
from lmfit.models import GaussianModel
import xgboost as xgb

In [7]:
os.environ["WANDB_API_KEY"] = '9c87219b80917d112c22c20a605d8dc6d6befdfd'

In [8]:
def get_time_cell_3x3(row):
    time = row[:9]
    energy = row[9:]
    return (time * energy).sum() / (energy.sum() + 1e-6)


def get_time_cell_5x5(row):
    time = row[:25]
    energy = row[25:]
    return (time * energy).sum() / (energy.sum() + 1e-6)

In [9]:
def get_new_feats_and_train_feats(df, layers=[0], conv_check=False):
    df['t_ECAL'] = df.timing.values

    for layer in layers:
        df['t{}_weighted_3x3'.format(layer)] = np.apply_along_axis(get_time_cell_3x3, 
                                                    axis=1, 
                                                    arr=df[['t{}_{}'.format(layer,c) for c in [6,7,8,11,12,13,16,17,18]] + \
                                                           ['l{}_{}'.format(layer,c) for c in [6,7,8,11,12,13,16,17,18]]].values)

        df['t{}_weighted_5x5'.format(layer)] = np.apply_along_axis(get_time_cell_5x5, 
                                                    axis=1, 
                                                    arr=df[['t{}_{}'.format(layer,c) for c in range(25)] + \
                                                           ['l{}_{}'.format(layer,c) for c in range(25)]].values)


    for layer in layers:
        df['l{}_sum_3x3'.format(layer)] = \
            df[['l{}_{}'.format(layer, i) for i in [6,7,8,
                                                    11,12,13,
                                                    16,17,18]]].values.sum(axis=1)

        df['l{}_sum_5x5'.format(layer)] = \
            df[['l{}_{}'.format(layer, i) for i in range(25)]].values.sum(axis=1)
        
    train_features = []
    
    if not conv_check:
        train_features += ['t0_{}'.format(i) for i in range(25)] + \
                          ['t0_weighted_3x3','t0_weighted_5x5']
    
    for layer in layers:
            train_features += ['l{}_{}'.format(layer, i) for i in range(25)]
            if not conv_check:
                train_features += ['l{}_sum_3x3'.format(layer)] + \
                ['l{}_sum_5x5'.format(layer)]
    
    return df, train_features

In [10]:
class MyDataset(Dataset):
    def __init__(self, X_, y_):
        self.X = torch.tensor(X_.values,dtype=torch.float32)
        self.y = torch.tensor(y_.values,dtype=torch.float32)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index],self.y[index]

class MyConvDataset1Layer(Dataset):
    def __init__(self, X_, y_):
        super(Dataset, self).__init__()
        self.X = torch.tensor(X_.values,dtype=torch.float32)
        self.y = torch.tensor(y_.values,dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index].reshape((5, 5)).unsqueeze(0), self.y[index]

class MyConvDataset2Layer(Dataset):
    def __init__(self, X_, y_):
        super(Dataset, self).__init__()
        self.X = torch.tensor(X_.values,dtype=torch.float32)
        self.y = torch.tensor(y_.values,dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index].reshape((5, 10)).unsqueeze(0), self.y[index]

In [11]:
def train_and_validate(model, optimizer, criterion, metric, train_loader, val_loader,
                       num_epochs, device, scheduler=None, verbose=True):
    
    prev_met = 10e3


    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, running_metric = 0, 0
        
        pbar = tqdm(train_loader, desc=f'Training {epoch}/{num_epochs}') \
            if verbose else train_loader

        for X_batch, y_batch in pbar:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            predictions = model(X_batch)
            y_batch = y_batch.unsqueeze(1)
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()
            
            if scheduler is not None:
                scheduler.step()

            metric_value = metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
            running_loss += loss.item() * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            if verbose:
                pbar.set_postfix({'loss': loss, 'MSE': metric_value})
        

        train_loss = running_loss / len(train_loader.dataset)
        train_metric = running_metric / len(train_loader.dataset)

        model.eval()
        running_loss, running_metric = 0, 0
        pbar = tqdm(val_loader, desc=f'Validating {epoch}/{num_epochs}') \
            if verbose else val_loader

        for X_batch, y_batch in pbar:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            predictions = model(X_batch)
            y_batch = y_batch.unsqueeze(1)
            loss = criterion(predictions, y_batch)
            
            metric_value =  metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
            running_loss += loss.item() * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            if verbose:
                pbar.set_postfix({'loss': loss, 'MSE': metric_value})

        val_loss = running_loss / len(val_loader.dataset)
        val_metric = running_metric / len(val_loader.dataset)
        
        if val_metric < prev_met:
            print('better metric =', val_metric)
            prev_met = val_metric

        wandb.log({'train_loss': train_loss,
                    'val_loss':  val_loss,
                    'train_metric': train_metric,
                   'val_metric': val_metric})
        
        

    print(f'Validation MSE: {val_metric:.3f}')
    
    return train_metric, val_metric

In [12]:
def get_rmse(x, y):
    return np.nanmean(((x - y) ** 2.0)) ** 0.5

def get_rmse_metric(x, y, folds=5):
    if x.shape != y.shape:
        print('x.shape != y.shape')
        raise

    splits = np.array_split(np.arange(len(x)), folds)

    rmse = []
    for split in splits:
        rmse.append(get_rmse(x[split], y[split]))

    return np.nanmean(rmse), np.nanstd(rmse)

In [13]:
def extract_timing_resolution(df, save=True, title=None):
    cell_size = 1.515
    mod = GaussianModel()
    E_bins = [10000., 30000., 50000., 70000., 90000.]

    sel = (df.cell_size == cell_size)
    df = df[sel & (df.p_ECAL > E_bins[0]) & (df.p_ECAL < E_bins[-1])].reset_index(drop=True)
    target = df.t_pred
    n_stds = 5

    range_all = (-n_stds*np.std(target - df.t_ECAL), n_stds*np.std(target - df.t_ECAL))

    fig, ax = plt.subplots(1, len(E_bins)-1, figsize=(20,3))
    bins_plotting = (np.array(E_bins[:-1]) + np.array(E_bins[1:])) / 2.0
    sigmas = []
    sigmas_std = []

    for i in [0, 1, 2, 3]:
        mask = (df.p_ECAL >= E_bins[i]) & (df.p_ECAL < E_bins[i+1])
        bin_heights, bin_borders, _ = ax[i].hist(target[mask]/df.t_ECAL[mask] - 1, bins=30, \
             range=(-n_stds*np.std(target[mask]/df.t_ECAL[mask] - 1), n_stds*np.std(target[mask]/df.t_ECAL[mask] - 1)), color='C02')
        bin_centers = (bin_borders[:-1] + bin_borders[1:]) / 2.0
        pars = mod.guess(bin_heights, x = bin_centers)
        out = mod.fit(bin_heights, pars, x = bin_centers)
        x = np.linspace(bin_borders[0], bin_borders[-1], 200)
        ax[i].plot(bin_centers, out.best_fit, label='best fit', color='C03')
        ax[i].set_title(f'Bin {i} ({0.001*E_bins[i]}-{0.001*E_bins[i+1]} GeV, {len(df.p_ECAL[mask])} entries)')
        ax[i].set_xlabel(r'$t_{rec} - t_{gen}$ [ns]', fontsize=14)

        
        sigmas.append(out.params['sigma'].value)
        sigmas_std.append(out.params['sigma'].stderr)
    
    if title is not None:
        plt.title(title)
    if save:
        plt.savefig(f'plot_hist_{title}.png', dpi=300, bbox_inches='tight')
    plt.close()
        
    return sigmas, sigmas_std

In [14]:
def plot_rec_curve(sigmas, sigmas_std, save=True, title=None):
    fig, ax = plt.subplots(figsize=(10,10))

    x = [20, 40, 60, 80]

    x_ref = [2., 3., 4., 5., 20., 30., 50., 70., 100.]
    y_ref1 = [44.71,34.90,29.48,26.19,19.39,17.40,15.35,14.04,13.63]
    y_ref2 = [56.99,47.04,41.35,37.89,26.15,22.83,16.34,13.36,12.36]
    
    if len(sigmas) == 0 or len(sigmas_std) == 0 or None in sigmas or None in sigmas_std:
        return
    
    y_b = [i*100000 for i in sigmas]
    yerr_b = [i*100000 for i in sigmas_std]

    ax.set_xlabel(r'$E_{gen}$ [GeV]', fontsize=18)
    ax.set_ylabel('Time resolution [ps]', fontsize=18)

    
    ax.grid(True, which="both")

    ax.errorbar(x, y_b, yerr=yerr_b, fmt='o', c='C03', alpha=0.7, label='Current reconstruction')
    ax.plot(x_ref, y_ref1, 'g--', label="Reference 1")
    ax.plot(x_ref, y_ref2, 'b--', label="Reference 2")
    ax.legend(fontsize=16, loc='upper right')
    
    if title is not None:
        plt.title(title)
    
    if save:
        plt.savefig(f'plot_rec_{title}.png', dpi=300, bbox_inches='tight')
    plt.close()

In [4]:
df aggregate= pd.read_csv('../input/ecal-complex/1ph_2L.csv')
#df = pd.read_csv('../input/hse-ecal/converted_all.csv')
df, tr_feats = get_new_feats_and_train_feats(df, [0, 1], True)
df = df.fillna(df.mean())
df['p_ECAL'] = df['eKinetic'].values * 1000
df['t_pred'] = 0

X = df[tr_feats]
X = (X - X.mean()) / X.std()

X['p_ECAL'] = df['p_ECAL']
X['cell_size'] = df[['cell_size']]
X['t_ECAL'] = df['t_ECAL']

y = X['t_ECAL']

NameError: name 'get_new_feats_and_train_feats' is not defined

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=812)
X_train = X_train[tr_feats]
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=812)

In [21]:
TrainSet = MyDataset(X_train, y_train)
ValSet = MyDataset(X_val, y_val)
TestSet = MyDataset(X_test[tr_feats], y_test)

TrainLoader = DataLoader(TrainSet, batch_size=1024, pin_memory=True, num_workers=2)
ValLoader = DataLoader(ValSet, batch_size=1024, pin_memory=True, num_workers=2)
TestLoader = DataLoader(TestSet, shuffle=False, pin_memory=True, num_workers=2)

In [49]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=2, padding='same')
        self.bn = nn.BatchNorm2d(32)

        self.head = nn.Linear(32 * 8 * 8, 128)
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)

        out = self.head(out)
        return out.reshape((128, -1))

In [1]:
N_EPOCH = 30
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = ConvNet()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCH * len(TrainLoader))


metric_rmse = lambda a, b: mean_squared_error(a, b, squared=False)



train_and_validate(net, optimizer, criterion, metric_rmse, TrainLoader, ValLoader, N_EPOCH, device, scheduler, verbose=True)

In [35]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.layers = nn.ModuleList(
            [
                self.build_block(len(tr_feats), 128),
                self.build_block(128, 256),
                self.build_block(256, 128),
                self.build_block(128, 64)
            ]
        )
        
        self.skip_connections = nn.ModuleList(
            [
                nn.AdaptiveAvgPool1d(128),
                nn.AdaptiveAvgPool1d(256),
                nn.AdaptiveAvgPool1d(128),
                nn.AdaptiveAvgPool1d(64)
            ]
        )
        self.head = nn.Linear(64, 1)
        
    def build_block(self, in_feats, out_feats):
        return  nn.Sequential(
                    nn.Linear(in_feats, out_feats),
                    nn.BatchNorm1d(out_feats),
                    nn.ReLU())
        
        
    def forward(self, x):
        out = x

        for i in range(len(self.layers)):
            out = self.skip_connections[i](out) + self.layers[i](out)
            
        out = self.head(out)
        return out

In [36]:
NUM_EPOCHS = 25
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
best_model = None
best_metric = 1e3

    
def train_and_evaluate(model, params, trial):
    
    TrainLoader = DataLoader(TrainSet, batch_size=params['batch_size'], pin_memory=True, num_workers=2)
    ValLoader = DataLoader(ValSet, batch_size=params['batch_size'], pin_memory=True, num_workers=2)
    
    criterion = nn.MSELoss()
    metric = lambda a, b: mean_squared_error(a, b, squared=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS * len(TrainLoader))

    
    if torch.cuda.is_available():
        criterion = criterion.cuda()
        model = model.cuda()
    
    print(next(model.parameters()).device)
    
    for epoch in tqdm(range(NUM_EPOCHS)):
        model.train()
        running_loss, running_metric = 0, 0
        
        for X_batch, y_batch in TrainLoader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            
            predictions = model(X_batch)
            y_batch = y_batch.unsqueeze(1)
            loss = criterion(predictions, y_batch)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            
            metric_value = metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
            running_loss += loss.item() * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            train_loss = running_loss / len(TrainLoader.dataset)
            train_metric = running_metric / len(TrainLoader.dataset)
            
            
        with torch.no_grad():
            model.eval()
            running_loss, running_metric = 0, 0
            for X_batch, y_batch in ValLoader:

                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(X_batch)
                y_batch = y_batch.unsqueeze(1)
                loss = criterion(predictions, y_batch)
                metric_value =  metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
                running_loss += loss.item() * X_batch.shape[0]
                running_metric += metric_value * X_batch.shape[0]
                
        val_loss = running_loss / len(ValLoader.dataset)
        val_metric = running_metric / len(ValLoader.dataset)
        
        trial.report(val_metric, epoch)
        
        if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
                
    return val_metric

    

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'batch_size': trial.suggest_categorical('batch_size', [1024, 2048, 4096])
    }
    
    model = ResNet()
    rmse = train_and_evaluate(model, params, trial)
    print(trial.params)
    
    global best_model
    global best_metric
        
    if rmse < best_metric:
        best_metric = rmse
        best_model = model
        print('New best')

    return rmse

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=20)

print('BEST PARAMETERS:')
print(study.best_params)

[I 2023-05-18 01:27:57,796] A new study created in memory with name: no-name-95436b2f-0f22-45ee-9332-5a5a1977300c


cpu


  0%|          | 0/25 [00:17<?, ?it/s]
[W 2023-05-18 01:28:15,418] Trial 0 failed with parameters: {'learning_rate': 0.08823686721298425, 'batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_27/1280475857.py", line 83, in objective
    rmse = train_and_evaluate(model, params, trial)
  File "/tmp/ipykernel_27/1280475857.py", line 38, in train_and_evaluate
    loss.backward()
  File "/opt/conda/lib/python3.7/site-packages/torch/_tensor.py", line 488, in backward
    self, gradient, retain_graph, create_graph, inputs=inputs
  File "/opt/conda/lib/python3.7/site-packages/torch/autograd/__init__.py", line 199, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt
[W 2023-05-18 01:28:15,423] Trial 0 failed w

KeyboardInterrupt: 

In [ ]:
N_EPOCH = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net = ResBlock()
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCH * len(TrainLoader), verbose=True)


metric_rmse = lambda a, b: mean_squared_error(a, b, squared=False)


run_config = {
    "name" : "ResNet",
    "learning_rate": 1e-3,
    "epochs": N_EPOCH
}

run = wandb.init(project="HSE_ECAL_project", config=run_config, name='ResNet Real')

train_and_validate(net, optimizer, criterion, metric_rmse, TrainLoader, ValLoader, N_EPOCH, device, scheduler, verbose=True)
run.finish()
best_model = net

In [22]:
NUM_EPOCHS = 30
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
best_model = None
best_metric = 1e3

def build_model(trial, params):
    in_feats = len(tr_feats)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    layers = []
    
    for i in range(num_layers):
        out_feats = trial.suggest_int('hidden_size_l{}'.format(i), 64, 256)
        
        layers.append(nn.Linear(in_features=in_feats, out_features=out_feats))
        layers.append(nn.BatchNorm1d(out_feats))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(p=params['dropout_p']))
        
        in_feats = out_feats
    
    layers.append(nn.Linear(in_features=in_feats, out_features=32))
    layers.append(nn.BatchNorm1d(32))
    layers.append(nn.ReLU())
    layers.append(nn.Linear(in_features=32, out_features=1))
    
    return nn.Sequential(*layers)
        
    
def train_and_evaluate(model, params, trial):
    criterion = nn.MSELoss()
    metric = lambda a, b: mean_squared_error(a, b, squared=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    
    if torch.cuda.is_available():
        criterion = criterion.cuda()
        model = model.cuda()
    
    print(next(model.parameters()).device)
    
    for epoch in tqdm(range(NUM_EPOCHS)):
        model.train()
        running_loss, running_metric = 0, 0
        
        for X_batch, y_batch in TrainLoader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            
            predictions = model(X_batch)
            y_batch = y_batch.unsqueeze(1)
            loss = criterion(predictions, y_batch)
            
            loss.backward()
            optimizer.step()
            
            metric_value = metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
            running_loss += loss.item() * X_batch.shape[0]
            running_metric += metric_value * X_batch.shape[0]
            train_loss = running_loss / len(TrainLoader.dataset)
            train_metric = running_metric / len(TrainLoader.dataset)
            
            
        with torch.no_grad():
            model.eval()
            running_loss, running_metric = 0, 0
            for X_batch, y_batch in ValLoader:

                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                predictions = model(X_batch)
                y_batch = y_batch.unsqueeze(1)
                loss = criterion(predictions, y_batch)
                metric_value =  metric(np.nan_to_num(predictions.cpu().detach().numpy()), np.nan_to_num(y_batch.cpu().detach().numpy()))
                running_loss += loss.item() * X_batch.shape[0]
                running_metric += metric_value * X_batch.shape[0]
                
        val_loss = running_loss / len(ValLoader.dataset)
        val_metric = running_metric / len(ValLoader.dataset)
        
        trial.report(val_metric, epoch)
        
        if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
                
    return val_metric

    

def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'dropout_p': trial.suggest_float('dropout_p', 0, 0.5)
    }
    
    model = build_model(trial, params)
    rmse = train_and_evaluate(model, params, trial)
    print(trial.params)
    
    global best_model
    global best_metric
        
    if rmse < best_metric:
        best_metric = rmse
        best_model = model
        print('New best')

    return rmse

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=25)
print('BEST PARAMETERS:')
print(study.best_params)

[I 2023-05-18 10:02:58,540] A new study created in memory with name: no-name-aa321c8c-7ace-4328-b82d-09699becadb3


cuda:0


  5%|▌         | 2/40 [00:28<09:00, 14.24s/it]
[W 2023-05-18 10:03:27,030] Trial 0 failed with parameters: {'learning_rate': 0.060850635313584846, 'dropout_p': 0.18163681413396549, 'num_layers': 2, 'hidden_size_l0': 114, 'hidden_size_l1': 176} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_23/3583032611.py", line 98, in objective
    rmse = train_and_evaluate(model, params, trial)
  File "/tmp/ipykernel_23/3583032611.py", line 44, in train_and_evaluate
    for X_batch, y_batch in TrainLoader:
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 628, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1316, in _next_data
    idx, data = self._get_data()
  File "/opt/conda/lib/python3.7/sit

KeyboardInterrupt: 

In [ ]:
NUM_EPOCHS = 400

model = nn.Sequential(
    nn.Linear(in_features=len(tr_feats), out_features=256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=256, out_features=256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=256, out_features=256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(in_features=256, out_features=128),
    nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Linear(in_features=128, out_features=1)
)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
model = model.to(device)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=40, eta_min=1e-4)

metric_rmse = lambda a, b: mean_squared_error(a, b, squared=False)

run_config = {
    "learning_rate": 1e-3,
    "epochs": NUM_EPOCHS,
    "optimizer": optimizer,
    "scheduler": 'None'
}

run = wandb.init(project="HSE_ECAL_project", config=run_config)

train_and_validate(model, optimizer, criterion, metric_rmse, TrainLoader, ValLoader, NUM_EPOCHS, device, scheduler, verbose=True)
run.finish()

In [ ]:
preds = []

for data, target in TestLoader:
    data = data.to(device)
    preds.append(best_model(data).item())

preds = np.array(preds)


mean_RMSE, std_RMSE = get_rmse_metric(y_test.values, preds)
print('mean_RMSE t_pred_NN: {:.5f}, std_RMSE: {:.5f}'.format(mean_RMSE, std_RMSE))


rec_title = f'ResNet_optuna_best_P1'
X_test['t_pred'] = preds
sigmas, sigmas_std = extract_timing_resolution(X_test, save=True, title=rec_title)
plot_rec_curve(sigmas, sigmas_std, save=True, title=rec_title)